In [1]:
pip install CausalBootstrapping

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install graphviz

Note: you may need to restart the kernel to use updated packages.


In [3]:
import causalBootstrapping as cb

In [4]:
from distEst_lib import MultivarContiDistributionEstimator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.metrics import classification_report

In [ ]:
causal_graph = '"Alcohol Consumption Causal Graph"; \
                D; F; S; A; P; M; N; J; K; \
                F -> D; \
                S -> D; \
                A -> D; \
                P -> D; \
                M -> D; \
                N -> D; \
                J -> D; \
                K -> D; \
                S -> F; \
                A -> F; \
                P -> F; \
                M -> F; \
                N -> F; \
                J -> F; \
                K -> F; \
                M -> J; \
                N -> K; '
# Dalc(D); Famrel(F); Sex(S); Age(A); Pstatus(P); Medu(M); Fedu(N); Mjob(J); Fjob(K)

In [ ]:
weight_func_lam, weight_func_str = cb.general_cb_analysis(causal_graph = causal_graph, 
                                                          effect_var_name = 'D', 
                                                          cause_var_name = 'S',
                                                          info_print = True)

# 此代码应按如下方式输出：

# Interventional prob.:p_{Y}(X)=\sum_{U,Z,Y'}[p(X|U,Z,Y')p(Z|U,Y)p(U,Y')]
# Causal bootstrapping weights function: [P(U,Y')P(U,Y,Z)]/N*[P(U,Y',Z)P(U,Y)]
# Required distributions:
# 1: P(U,Y')
# 2: P(U,Y,Z)
# 3: P(U,Y',Z)
# 4: P(U,Y)